## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [2]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds

# Import code
from brats.load_data import load_data

# Performance 
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import seaborn


# Pipeline and gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

#preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# scaler
from sklearn.preprocessing import RobustScaler

#Machine learning classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import ShuffleSplit

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor



C:\Users\hilde\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


#### Load data

In [3]:
data_brats = load_data()

# Convert to dataframe
X = pd.DataFrame(data_brats)

# Print data 
print(f'The number of samples in data_brats: {len(X.index)}')
print(f'The number of columns in data_brats: {len(X.columns)}')

The number of samples in data_brats: 167
The number of columns in data_brats: 725


c:\Users\hilde\Python\TM10007_Machine_Learning\TM10007\brats\load_data.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


#### Split data in X and y
Split in X (data) and y (label)

In [4]:
# split column label from dataset X
y = X.pop('label')

#### Split data in train and test set
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [5]:
# Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)  


#### Preprocessing : 

In [6]:
# infinity to NaN
X_train[X_train==np.inf]=np.nan

# non-numeric features to NaN
X_train = X_train.replace(['#DIV/0!'], np.nan)
X_train = X_train.apply(pd.to_numeric, errors='coerce')

X_test  = X_test.replace(['#DIV/0!'], np.nan)
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
X_train_drop = X_train.dropna(axis=1, thresh=min_count)
X_labels = X_train_drop.keys()

X_test_drop = X_test[X_labels]
print(X_test_drop.shape)
print(X_train_drop.shape)

# fill the NaN observations.
data_fill_train = X_train_drop.fillna(X_train_drop.mean()) 
data_fill_test = X_test_drop.fillna(X_test_drop.mean()) 

(34, 704)
(133, 704)


# Pipeline
##### building a pipeline to define each transformer type

In [7]:
numeric_transformer = Pipeline(steps=[
    ('imputation', SimpleImputer(missing_values = np.NaN, strategy='most_frequent')),   # kan ook strategy = 'most frequent'
    ('robust', RobustScaler())])


In [8]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
pca = decomposition.PCA()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features), ('pca', pca)]) 

#preprocessor = ColumnTransformer(
 #   transformers=[
  #      ('num', numeric_transformer, numeric_features)])



#### Loop some classifiers and check performance


In [11]:
# Beste classifier gebruiken! Dit gebruiken om keuze te onderbouwen
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()]
    
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(data_fill_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(data_fill_test, y_test))

ValueError: not enough values to unpack (expected 3, got 2)

#### Hyperparametersearch

In [ ]:
# Hyperparameter suggestions
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8]}                        # kan ook nog, maar deed het niet: 'classifier__criterion' :['gini', 'entropy']
from sklearn.model_selection import GridSearchCV

# Gridsearch with 5-fold cross validation
Gridsearch_CV = GridSearchCV(pipe, param_grid, n_jobs= 1, cv=5)   # Hier kunnen we kiezen voor GridSearh of randomgridsearch
                  
Gridsearch_CV.fit(X_train, y_train)  

#Kijken welke parameters het best zijn en die uiteindelijk gebruiken!
print(Gridsearch_CV.best_params_)    
print(Gridsearch_CV.best_score_)

{'classifier__max_depth': 7, 'classifier__max_features': 'sqrt', 'classifier__n_estimators': 500}
0.8951566951566953


In [ ]:
#create pipeline with best parameters and best classifier : voor nu gekozen voor RF 
# Error bij andere classifiers en niet bij xgb?  .XGBClassifier opzoeken bij RF enzo
hyperparams_after_gridsearch= Gridsearch_CV.best_params_
params_after_grid = { **static_params, **hyperparams_after_gridsearch}
pipe_after_grid = Pipeline([('classifier', xgb.XGBClassifier(**params_after_grid))])

#fit pipe with hyperparameters on complete train set
bst= pipe_after_grid.fit(X_train, y_train)

NameError: name 'static_params' is not defined

In [ ]:
score_train = roc_auc_score(y_train, bst.predict_proba(X_train)[:, 1])
print(score_train)

#TEST WERKT NOG NIET
#score_test = roc_auc_score(y_test, bst.predict_proba(X_test)[:, 1])
# print(score_test)


1.0
